# Facebook Check-in Challenge

In [3]:
#Starter code from Kaggle

import pandas as pd
import numpy as np

print 'reading train.csv'
train = pd.read_csv('train.csv', nrows=1000)
print 'reading test.csv'
test = pd.read_csv('test.csv')
start_time = time.time()

reading train.csv
reading test.csv


In [9]:
print train.columns

Index([u'row_id', u'x', u'y', u'accuracy', u'time', u'place_id'], dtype='object')


In [13]:
print test.columns

Index([u'row_id', u'x', u'y', u'accuracy', u'time', u'hour', u'weekday',
       u'month', u'year'],
      dtype='object')


In [10]:
size = 10.0;

x_step = 0.2
y_step = 0.2

x_ranges = zip(np.arange(0, size, x_step), np.arange(x_step, size + x_step, x_step));
y_ranges = zip(np.arange(0, size, y_step), np.arange(y_step, size + y_step, y_step));

In [12]:
#Calculating individual time metrics from the time column
print('Calculate hour, weekday, month and year for train and test')
train['hour'] = (train['time']//60)%24+1 # 1 to 24
train['weekday'] = (train['time']//1440)%7+1
train['month'] = (train['time']//43200)%12+1 # rough estimate, month = 30 days
train['year'] = (train['time']//525600)+1 

test['hour'] = (test['time']//60)%24+1 # 1 to 24
test['weekday'] = (test['time']//1440)%7+1
test['month'] = (test['time']//43200)%12+1 # rough estimate, month = 30 days
test['year'] = (test['time']//525600)+1

X_train = train[['x','y','accuracy','time', 'hour', 'weekday', 'month', 'year']];
y_train = train[['place_id']];
X_test = test[['x','y','accuracy','time', 'hour', 'weekday', 'month', 'year']];
X_test_labels = test[['row_id']];

Calculate hour, weekday, month and year for train and test


In [ ]:
print 'training classifier'
clf = RandomForestClassifier(n_estimators = 10, n_jobs = -1,random_state=0);
clf.fit(X_train, y_train)

print 'generating predictions'
preds = dict(zip([el for el in clf.classes_], zip(*clf.predict_proba(X_test))))
preds = pd.DataFrame.from_dict(preds)

prihree most lkel
preds['0_'], preds['1_'], preds['2_'] = zip(*preds.apply(lambda x: preds.columns[x.argsort()[::-1][:3]].tolist(), axis=1));
preds = preds[['0_','1_','2_']];
preds['row_id'] = X_test_labels
preds['place_id'] = preds[['0_', '1_', '2_']].apply(lambda x: ' '.join([str(x1) for x1 in x]), axis=1)

preds.drop('0_', axis=1, inplace=True)
preds.drop('1_', axis=1, inplace=True)
preds.drop('2_', axis=1, inplace=True)
print 'writing to file'
sub_file = os.path.join('rf_submission_' + str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + '.csv')
preds.to_csv(sub_file,index=False)
print 'finished'